# Importar librerías

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns



# Cargar el dataset

In [ ]:
movies_file = "dataset/movies.csv"
ratings_file = "dataset/ratings.csv"
ratings_df = pd.read_csv(ratings_file)
movies_df = pd.read_csv(movies_file)

DataFrame de los registros de calificaciones("ratings.csv")

In [ ]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


DataFrame de los registros de películas("movies.csv")

In [ ]:
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


# Normalizar los DataFrames
Combinar el dataframe 'ratings_df' y 'movies_df' concatenados por el valor de la columna 'movieId'

In [ ]:
df = ratings_df.merge(movies_df,on='movieId', how='left')
df.head()

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,964981247,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,964982224,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,964983815,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,964982931,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


# Procesamiento de datos

Convertir Géneros en fila a columnas. Se crea una nueva columna para cada género. Por cada género existente se crea una columna a la que inicialmente se le asignarán valores de 0

In [ ]:
#Columna géneros de películas
x = df.genres 
a = list()
for i in x:
    #Dividir la cadena de texto en una lista
    a.append(i.split('|'))
#Crear columnas    
a = pd.DataFrame(a)   
b = a[0].unique()
for i in b:
    #Asignar valor 0 a cada columna
    df[i] = 0  
df.head(1)

,userId,movieId,rating,timestamp,title,genres,Adventure,Comedy,Action,Mystery,Crime,Thriller,Drama,Animation,Children,Horror,Documentary,Sci-Fi,Fantasy,Film-Noir,Western,Musical,Romance,(no genres listed),War
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Se asigna 1 al valor de la columna del género que pertenece a cada película.

In [ ]:
for i in b:
    df.loc[df['genres'].str.contains(i), i] = 1
df.head(1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  


,userId,movieId,rating,timestamp,title,genres,Adventure,Comedy,Action,Mystery,Crime,Thriller,Drama,Animation,Children,Horror,Documentary,Sci-Fi,Fantasy,Film-Noir,Western,Musical,Romance,(no genres listed),War
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,1,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0


# Verificar y Limpiar Datos
Comprobar si existen valores NaN en el DataFrame

In [ ]:
df.isna().sum()

userId                0
movieId               0
rating                0
timestamp             0
title                 0
genres                0
Adventure             0
Comedy                0
Action                0
Mystery               0
Crime                 0
Thriller              0
Drama                 0
Animation             0
Children              0
Horror                0
Documentary           0
Sci-Fi                0
Fantasy               0
Film-Noir             0
Western               0
Musical               0
Romance               0
(no genres listed)    0
War                   0
dtype: int64

Limpiar la fila inconsisitente y verificar nuevamente

In [ ]:
df.dropna(inplace = True)
df.isnull().sum()

userId                0
movieId               0
rating                0
timestamp             0
title                 0
genres                0
Adventure             0
Comedy                0
Action                0
Mystery               0
Crime                 0
Thriller              0
Drama                 0
Animation             0
Children              0
Horror                0
Documentary           0
Sci-Fi                0
Fantasy               0
Film-Noir             0
Western               0
Musical               0
Romance               0
(no genres listed)    0
War                   0
dtype: int64

In [ ]:
df.head()

,userId,movieId,rating,timestamp,title,genres,Adventure,Comedy,Action,Mystery,Crime,Thriller,Drama,Animation,Children,Horror,Documentary,Sci-Fi,Fantasy,Film-Noir,Western,Musical,Romance,(no genres listed),War
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,1,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0
1,1,3,4.0,964981247,Grumpier Old Men (1995),Comedy|Romance,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,1,6,4.0,964982224,Heat (1995),Action|Crime|Thriller,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,47,5.0,964983815,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,50,5.0,964982931,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0


# Eliminar caracteristicas que no influyen en el modelo
Se procede a eliminar 2 características que no se utilizarán.

**timestamp.-** Es el valor de fecha, que no se toma en cuenta para el sistema de recomendación.

**genres.-** Es el valor que indica los géneros a los cuales pertenecen las películas. No se utiliza ya que cada género se encuentra en columnas. 

**title.-** No se elimina del DataFrame, ya que tendrá utilidad para relacionar los resultados de la recomendación con el título de cada película.

In [ ]:
df = df.drop(['timestamp','genres','(no genres listed)'],axis =1) #Eliminar características 
df.head(2)

,userId,movieId,rating,title,Adventure,Comedy,Action,Mystery,Crime,Thriller,Drama,Animation,Children,Horror,Documentary,Sci-Fi,Fantasy,Film-Noir,Western,Musical,Romance,War
0,1,1,4.0,Toy Story (1995),1,1,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0
1,1,3,4.0,Grumpier Old Men (1995),0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


# Método de ingeniería de características 

In [ ]:
min_rating = min(df["rating"])
max_rating = max(df["rating"])

df = df.sample(frac=1, random_state=42)

# Conjunto original de los datos
# Crear el emparejamiento tradicional (x, y) 
#X = df[["userId", "movieId"]]

X = df[['userId', 'movieId', 'Adventure', 'Comedy', 'Action', 'Mystery', 'Crime', 'Thriller',   
        'Drama', 'Animation', 'Children', 'Horror', 'Documentary', 'Sci-Fi', 'Fantasy', 'Film-Noir', 'Western', 'Musical', 'Romance']]

# Normalización Min-Max. 
y = df['rating'].apply(lambda x: (x - min_rating) / (max_rating - min_rating)).values
y = y.astype('int')# Arregla el error continous



# Dividir el dataset en training and testing 

In [ ]:
#Dividir los datos en conjuntos de datos de entrenamiento y prueba.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
print("Datos de entrenamiento",X_train.shape, y_train.shape)
print("Datos de prueba",X_test.shape, y_test.shape)

Datos de entrenamiento (70585, 19) (70585,)
Datos de prueba (30251, 19) (30251,)


# Selección de características
Mediante una técnica de selección de características, se puede obtener aquellas que son importantes para los modelos de SVD y KNN. El modelo de K-means utilizará carácteristicas distintas, las cuales son los géneros de las películas que ya encuentran en columnas, y como valor a salida  se tomará el rating.


In [ ]:
from sklearn.ensemble import RandomForestClassifier
RF = RandomForestClassifier(max_depth=7, n_estimators=100)
RF.fit(X_train, y_train)
y_pred_RF= RF.predict(X_test)

df_all = pd.DataFrame()

cols = ['Machine Learning Classification Method',
        'Train Accuracy', 
        'Test Accuracy', 
        "Top 1 Feature Predictor",
        "Top 2 Feature Predictor",
        "Top 3 Feature Predictor",
        "Top 4 Feature Predictor",
        "Top 5 Feature Predictor"]

df_all.loc['Machine Learning Classification Method', 
       'Details'] = 'Random Forest Classifier'
df_all.loc['Train Accuracy', 'Details'] = RF.score(X_train, y_train) 
df_all.loc['Test Accuracy', 'Details'] = RF.score(X_test, y_test)

feature_importances = X.columns[np.argsort(RF.feature_importances_)][-3:]

df_all.loc['Top 1 Feature Predictor', 'Details'] = feature_importances[2]
df_all.loc['Top 2 Feature Predictor', 'Details'] = feature_importances[1]
df_all.loc['Top 3 Feature Predictor', 'Details'] = feature_importances[0]


display(df_all)

,Details
Machine Learning Classification Method,Random Forest Classifier
Train Accuracy,0.868697
Test Accuracy,0.870351
Top 1 Feature Predictor,movieId
Top 2 Feature Predictor,userId
Top 3 Feature Predictor,Drama


# Generar un archivo plano con el dataset generado despúes haber aplicado el proceso de cleasing o limpiado de dato. 

In [ ]:
#Exportar DataFrame a CSV
ruta = "rating_movies.csv"
df.to_csv(ruta)
df.head()

,userId,movieId,rating,title,Adventure,Comedy,Action,Mystery,Crime,Thriller,Drama,Animation,Children,Horror,Documentary,Sci-Fi,Fantasy,Film-Noir,Western,Musical,Romance,War
67037,432,77866,4.5,Robin Hood (2010),1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1
42175,288,474,3.0,In the Line of Fire (1993),0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
93850,599,4351,3.0,Point Break (1991),0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0
6187,42,2987,4.0,Who Framed Roger Rabbit? (1988),1,1,0,1,1,0,0,1,1,0,0,0,1,0,0,0,0,0
12229,75,1610,4.0,"Hunt for Red October, The (1990)",1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


# Preguntas

## Qué modelo de ML/DL/DM usted aplicará a su proyecto para ser resuelto?

Para realizar la recomendación de películas utilizaré Singular Value Decomposition(SVD) aplicado al filtrado colaborativo.

## Con qué modelos comparará ese modelo y por qué?

SVD como modelo principal, será comparado con K-Nearest-Neighbor(KNN) y K-means, debido a que se obtiene los mismos resultados que son la recomendación de películas. El punto de comparación se evaluará de acuerdo a las métricas del  modelo. Estos algoritmos serán utilizados por que son aplicado en las técnicas de Filtrado colaborativo y Clustering., las cuales se utilizan en la mayoria de sistemas de recomendación.

## Cuáles son los hyperparameters que utilizará para cada modelo? Interprete los resultados. 

Los hiperparámetros que el modelo SVD necesita para llevar acabo el modelo son ‘n_epochs’ , ‘lr_al’ y ‘reg_all’. El primer
hiperparámetro ‘n_epochs’ representa el número de iteracciones del procedimiento. El segundo hiperparámetro lr_all representa el valor de la tasa de aprendizaje para todos los parámetros.  El tercer hiperparámetro reg_all representa la asignación del término de regularización para todos los parámetros. 

Los hiperparámetros que el modelo KNN necesita para llevar acabo el modelo son ‘n_neighbors’ y ‘weights’. El primer hiperparámetro ‘n_neighbors’ representa el número de vecinos a utilizar para el procedimiento. El segundo hiperparámetro es weights y representa funciones para evaluar los pesos que
proporcionan el tratamiento con la parte inicial. 


Los hiperparámetros que el modelo K-means necesita para llevar acabo el modelo son ‘n_clusters’ e ‘init’. El primer hiperparámetro ‘n_clusters’ representa el numero de cluster a aplicar. El segundo hiperparámetro es 'init', que representa el método de inicialización.'

## Qué métricas utilizará para evaluar su modelo? Comente sobre cada métrica y qué valor espera.

Para evaluar los modelos se utilizará la métrica RMSE para estimar la tasa de error al predecir un elemento. Otra métrica a evaluar es el Accurancy que indicará el valor de la exactitud del modelo en la recomendación de las películas, de la cual se espera que sea mayor al 81%.

